# Eigenvalue problem

In [50]:
import numpy as np
np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)
import math

### Initial tests:

Given the matrix A above:

In [51]:
A = np.array([[3, 4, 0],
              [4, 3, 0],
              [0, 0, 2]])

Solving the characteristic equation `det(A - λI) = 0` we get the following eigenvalues:

In [40]:
lambda1 = -1
lambda2 = 2
lambda3 = 7

Then, replacing those eigenvalues in `Ax = λx` we get the following normalized associated eigenvectors:

In [41]:
# associated with lambda1 = -1
x1 = np.array([0.707,
              -0.707,
               0.])


# associated with lambda2 = 2
x2 = np.array([0.,
               0.,
               1.])

# associated with lambda3 = 7
x3 = np.array([0.707,
               0.707,
               0.])


Now that we have the exact solution, we'll use the QR method to find an approximate solution by leveraging the Householder transformations. But first, let's define some helper functions:

In [42]:
def multiply_by_scalar(matrix, scalar):
    result = np.zeros(shape=dimension_of(matrix))
    for i in range(len(matrix)):
        for j in range(len(matrix[0])):
            result[i][j] = scalar * matrix[i][j]
    return result    

def scalar_product(a, b):
    result = 0
    for i in range(len(a[0])):
        result += a[0][i]*b[0][i]
    return result

def multiply_matrixes(a, b):
    m,n = dimension_of(a)
    _,p = dimension_of(b)
    result = np.zeros(shape=(m,p))
    for i in range(m):
        for j in range(p):
            for k in range(n):
                result[i][j] += a[i][k] * b[k][j]
    return result                          

def nth_identity(n):
    return np.identity(n)

def sum_matrixes(a, b):
    result = np.zeros(shape=dimension_of(a))
    for i in range(len(a)):
        for j in range(len(a[0])):
            result[i][j] += a[i][j] + b[i][j]
    return result
    
def dimension_of(matrix):
    return matrix.shape

def nth_canonical_form(n, dimension):
    zeros = np.zeros((1,dimension[1]))
    zeros[0][n-1] = 1
    return zeros

def norm(v):
    sum_square = 0
    for i in v[0]:
        sum_square += math.pow(i, 2)
    return math.sqrt(sum_square)

def compute_sigma(a): # by convention we're using the signal of first element
    if a[0][0] >= 0:
        return 1
    return -1

def compute_vi(A, i): # computes vi for finding the householder matrix
    ai = np.array([A[:, i - 1]])
    ei = nth_canonical_form(i, dimension_of(A))
    sigma = compute_sigma(ai)
    return sum_matrixes(ai, multiply_by_scalar(ei, sigma*(norm(ai))))

def compute_hi(A, i): # computes householder matrix for a given index
    vi = compute_vi(A, i)
    scalar = -2/scalar_product(vi,vi)
    matrix = multiply_matrixes(vi.transpose(), vi)
    identity = nth_identity(dimension_of(A)[0])
    return sum_matrixes(identity, multiply_by_scalar(matrix, scalar))

def compute_h_list(A): # computes all householder matrixes from i = 1 to i = n-1
    Ai = np.copy(A)
    result = []
    for i in range(1, len(A)):
        Hi = compute_hi(Ai, i)
        result.append(Hi)
        Ai = multiply_matrixes(Hi, Ai)
    return result

def compute_qi(H_list):
    Q = nth_identity(len(H_list[0]))
    for i in range(len(H_list)):
        Q = multiply_matrixes(Q, H_list[i])
    return Q
        
def compute_ri(H_list, A):        
    R = nth_identity(len(A))
    for i in range(len(H_list)):
        R = multiply_matrixes(R, H_list[len(H_list) - i -1])        
    return multiply_matrixes(R, A)

Now, we can define the QR factorization function as below:

In [43]:
def qr_factorization(A, iterations):
    Ai = np.copy(A)
    Qi = nth_identity(dimension_of(A)[0])
    Vi = np.copy(Qi)
    for i in range(1,iterations+1):
        H_list = compute_h_list(Ai)       
        Ri = compute_ri(H_list, Ai)
        Qi = compute_qi(H_list)
        Vi = multiply_matrixes(Vi, Qi)
        Ai = multiply_matrixes(Ri, Qi)
    return Ai, Vi

In [44]:
result = qr_factorization(A, 100)
print(result[0]) # matrix where the main diagonal elements are the eigenvalues
print(result[1]) # matrix where the columns are the eigenvectors (if A is symetric)

[[-1.  0.  0.]
 [ 0.  7.  0.]
 [ 0.  0.  2.]]
[[-0.707 -0.707  0.   ]
 [ 0.707 -0.707  0.   ]
 [ 0.     0.     1.   ]]


We can see that the main diagonal elements of the first matrix correspond to the eigenvalues calculated beforehand (-1, 2 and 7) and the columns in the second matrix matches the eigenvectors previously calculated as well.

Now, let's take another test, this time with a non symetric matrix:

In [127]:
A = np.array([[3, 4, 0],
              [1, 3, 0],
              [0, 0, 2]])

Now, solving the eigenvalue and eigenvector equations, we get the following:

In [128]:
# eigenvalues
lambda1 = 1.
lambda2 = 2.
lambda3 = 5.

# eigenvectors
x1 = np.array([ 0.894,
               -0.447,
                0.])

x2 = np.array([ 0.,
                0.,
                1.])

x3 = np.array([ 0.894,
                0.447,
                0.])

In [131]:
result = qr_factorization(A, 100)
print(result[0])
print(result[1])

[[ 1. -0.  0.]
 [-3.  5.  0.]
 [ 0.  0.  2.]]
[[-0.447 -0.894  0.   ]
 [ 0.894 -0.447  0.   ]
 [ 0.     0.     1.   ]]


The  result above show us that the eigenvalues are computed correctly, however, the eigenvectors don't converge. (because A is no longer symetric)

Lets take another case of study:

In [53]:
A = np.array([[ 1, 1],
              [-3, 1]])

Notice that A eigenvalues are not real, but complex numbers. Thus, the algorithm does not converge (you can play with the number of iterations and see it not converging):

In [61]:
result = qr_factorization(A, 100)
print(result[0])
print(result[1])

[[ 1.6  2.8]
 [-1.2  0.4]]
[[-0.316 -0.949]
 [ 0.949 -0.316]]


In the last initial test, let's consider the following matrix:

In [62]:
A = np.array([[ 3.   , 3.],
              [ 0.333, 5.]])

Again, solving the equations we get the following eigenvalues and eigenvectors:

In [64]:
# eigenvalues
lambda1 = 2.586
lambda2 = 5.414

# eigenvectors
x1 = np.array([ 0.991,
               -0.137])

x2 = np.array([ 0.779,
                0.627])

And our algorithm give us the following result:

In [76]:
result = qr_factorization(A, 100)
print(result[0])
print(result[1])

[[5.414 2.667]
 [0.    2.586]]
[[ 0.779 -0.627]
 [ 0.627  0.779]]


## Computational tasks

Let's consider the following matrix representing a vibration system composed of 5 masses:

In [120]:
alfa1= 1
alfa2= 1
alfa3= 1
alfa4= 1
alfa5= 1

A = np.array([[ -2*alfa1,    alfa1,       0.,       0.,      0.],
              [    alfa2, -2*alfa2,    alfa2,       0.,       0.],
              [       0.,    alfa3, -2*alfa3,    alfa3,       0.],
              [       0.,        0.,   alfa4, -2*alfa4,    alfa4],
              [       0.,        0.,       0.,   alfa5, -2*alfa5]])

In [121]:
result = qr_factorization(A, 1000)
print(result[0])
print(result[1])

[[-1.     0.    -0.     0.     0.   ]
 [ 0.    -2.     0.    -0.    -0.   ]
 [-0.     0.    -3.    -0.    -0.   ]
 [ 0.    -0.    -0.    -3.732  0.   ]
 [-0.    -0.    -0.    -0.    -0.268]]
[[-0.5    0.577  0.5   -0.289  0.289]
 [-0.5   -0.    -0.5    0.5    0.5  ]
 [ 0.    -0.577 -0.    -0.577  0.577]
 [ 0.5    0.     0.5    0.5    0.5  ]
 [ 0.5    0.578 -0.5   -0.289  0.289]]


Last, let's take the following matrix:

In [122]:
A = np.array([[ -2,   1,   0,  0,  0,  0,  0],
              [  1, -11,  10,  0,  0,  0,  0],
              [  0,  10, -11,  1,  0,  0,  0],
              [  0,   0,   1, -2,  1,  0,  0],
              [  0,   0,  0,   1, -2,  1,  0],
              [  0,   0,  0,   0,  1, -2,  1],
              [  0,   0,  0,   0,  0,  1, -2]])

In [126]:
result = qr_factorization(A, 50000)
print(result[0])
print(result[1])

[[ -0.603  -0.      0.     -0.     -0.     -0.     -0.   ]
 [  0.     -1.476  -0.     -0.     -0.     -0.      0.   ]
 [  0.     -0.     -2.272   0.     -0.     -0.     -0.   ]
 [  0.     -0.      0.     -2.785   0.      0.      0.   ]
 [  0.     -0.     -0.     -0.     -3.648   0.     -0.   ]
 [  0.      0.      0.     -0.     -0.    -21.053  -0.   ]
 [  0.      0.      0.      0.      0.     -0.     -0.164]]
[[-0.299  0.352  0.792 -0.322  0.06   0.037 -0.227]
 [-0.417  0.185 -0.215  0.253 -0.099 -0.706 -0.416]
 [-0.404  0.141 -0.267  0.24  -0.079  0.706 -0.428]
 [-0.027 -0.507 -0.178 -0.557  0.411 -0.037 -0.479]
 [ 0.366 -0.406  0.315  0.197 -0.599  0.002 -0.451]
 [ 0.538  0.294  0.093  0.402  0.575 -0.    -0.35 ]
 [ 0.386  0.56  -0.341 -0.513 -0.349  0.    -0.19 ]]
